# АНАЛИЗ ПРИЧИН УХОДА КЛИЕНТОВ ОТ БАНКА

Для анализа причин ухода клиентов необходимо ответить на вопросы. Для начала прочитаем данные и уберём лишние столбцы 

In [2]:
import pandas as pd

churn_data = pd.read_csv('data/churn.csv')
churn_data = churn_data.drop('RowNumber', axis=1) #убираем столбец RowNumber - в задании он не нужен

import plotly
import plotly.express as px

churn_data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Вопрос №1. 
Каково соотношение ушедших и лояльных клиентов? Покажите это на графике и дайте комментарий по соотношению.

In [3]:
pie_data = churn_data.groupby(by='Exited')[['Exited']].count() # считаем количество клиентов по признаку лояльности (1 - лояльные, 0 - ушедшие)

type_of_clients = {0:'Лояльные клиенты', 1:'Ушедшие клиенты'} # вводим словарь для легенды графика

#строим круговую диаграмму, чтобы показать структуру

fig_pie = px.pie(
    data_frame=pie_data, 
    names=type_of_clients,
    values='Exited',
    title='Структура клиентов банка (ушедшие/лояльные)',
    height=500, #высота
    width=500, #ширина
)
fig_pie.show()

## Вывод по Вопросу №1. 
Подавляющее большинство клиентов банка (почти 80%) - это лояльные клиенты. Соотношение количества ушедших и лояльных клиентов близко к 1:4 (20,4% к 79,6%)

## Вопрос №2. 
Постройте график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. Опишите распределение и сделайте выводы.

In [4]:
mask1 = churn_data['Balance'] > 2500 #делаем маску по размеру баланса

hist_data = churn_data[mask1]['Balance'] #делаем новый df с учётом маски

fig_hist = px.histogram(
    data_frame=hist_data,
    x='Balance',
    nbins=20,
    histnorm='percent', #при необходимости можно добавить комментирование на строку для получения на оси количества клиентов вместо доли в %
    marginal = 'box',
    title='Распределение баланса пользователей, у которых на счету больше $2500'
)

fig_hist.update_xaxes(title_text='Баланс на всех счетах клиента', col=1, row=1) # добавляем подписи на оси 
fig_hist.update_yaxes(title_text='% от количества клиентов', col=1, row=1)


fig_hist.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True) #добавляем рамки
fig_hist.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)


fig_hist.show()

## Вывод по Вопросу №2. 
Среди клиентов банка со балансом на счёте больше $2500 медианное значение баланса на счёте клиента составляет $120к. Почти у половины клиентов баланс на счете находится в пределах от $100к до $140к (межквартильный интервал). В соответствии с границами распределения минимальное значение баланса составляет $41к, а максимальное - $198к. Экстремальные значения также встречаются: например, минимальное - $3,7к и $251к, но на них приходится менее 1% клиентов. 

Учитывая сказанное выше можно сказать, что банк ориентирован на предложение банковских продуктов под определённый баланс клиентов (80% клиентов с балансом счёта в диапазоне $80к - $160к).

## Вопрос №3. 
Посмотрите на распределение баланса клиента в разрезе признака оттока. Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Подумайте и напишите, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [14]:
box_data = churn_data.loc[::, ['Exited', 'Balance']] #создаем df для коробочной диаграммы, оставляя только нужные столбцы

box_data['Exited'] = box_data['Exited'].apply(lambda x: 'Ушедший клиент' if x==1 else 'Лояльный клиент') #преобразуем значения в столбце признак клиента


fig_box = px.box(
    data_frame=box_data,
    x='Balance',
    color='Exited',
    title='Распределение баланса пользователей по признаку оттока',
    labels={'Exited': 'Признак клиента'},
)

fig_box.update_xaxes(title_text='Баланс на всех счетах клиента', col=1, row=1) # добавляем подписи на оси 

fig_box.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True) #добавляем рамки
fig_box.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

fig_box.show()

## Вывод по Вопросу №3.
Среди ушедших клиентов отмечается более высокое медианное значение баланса на счете ($110к проти $92к), а также меньший разрброс значений в балансе на счёте. Исходя из этого можно сделать вывод, что эти клиенты - более состоятельные и нашли лучшие условия по размещению своих средств, пакету услуг и/или качеству обслуживания в другом банке.

## Вопрос №4. 
Посмотрите на распределение возраста в разрезе признака оттока. В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [16]:
box_data_age = churn_data.loc[::, ['Exited', 'Age']] #создаем df для коробочной диаграммы, оставляя только нужные столбцы

box_data_age['Exited'] = box_data_age['Exited'].apply(lambda x: 'Ушедший клиент' if x==1 else 'Лояльный клиент') #преобразуем значения в столбце признак клиента

fig_box_age = px.box(
    data_frame=box_data_age,
    x='Age',
    color='Exited',
    title='Распределение возраста пользователей по признаку оттока',
    labels={'Exited': 'Признак клиента'},
)

fig_box_age.update_xaxes(title_text='Возраст клиента', col=1, row=1) # добавляем подписи на оси 

fig_box_age.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True) #добавляем рамки
fig_box_age.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

fig_box_age.show()

## Вывод по Вопросу №4. 
Наибольшее количество потенциальных выбросов зафиксировано в группе лояльных клиентов. Границы распределения возраста в группе лояльных клиентов - 18 лет и 56 лет, а 50% клиентов в возрасте от 31 до 41 года, медианный возраст - 36 лет. В группе ушедших клиентов верхняя граница распределения возраста выше - 71 год. Также выше и медианный возраст (45 лет).\
С учётом этого стоит обратить внимание на работу с более "возрастной" аудиторией - от 56 до 70 лет (по ним идет отличие в выбросах), а также с аудиторией от 40 до 51 года (эта группа клиентов составляет значительную долю в ушедших клиентах, но при этом малую долю - в лояльных клиентах). Обе указанные группы являются клиентами со значительной финансовой подушкой с учетом периода жизни (зрелый и предпенсионный возраты) и потенциальной более доходной группой, что подтверждается графиком к Вопросу №3.

## Вопрос №5. 
Постройте график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты. Добавьте расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Если не видите явной взаимосвязи, укажите это.

In [23]:
churn_data.groupby(by='Exited')['Age'].mean()

Exited
0    37.408389
1    44.837997
Name: Age, dtype: float64

In [ ]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB
